In [1]:
!pip install mediapipe opencv-python numpy 


   ---------------------------------------- 0.0/50.8 MB ? eta -:--:--
   - -------------------------------------- 2.1/50.8 MB 10.7 MB/s eta 0:00:05
   --- ------------------------------------ 4.5/50.8 MB 11.2 MB/s eta 0:00:05
   ----- ---------------------------------- 6.6/50.8 MB 10.6 MB/s eta 0:00:05
   ------ --------------------------------- 8.7/50.8 MB 10.7 MB/s eta 0:00:04
   -------- ------------------------------- 11.0/50.8 MB 10.9 MB/s eta 0:00:04
   ---------- ----------------------------- 13.6/50.8 MB 11.0 MB/s eta 0:00:04
   ------------ --------------------------- 16.0/50.8 MB 11.1 MB/s eta 0:00:04
   -------------- ------------------------- 18.6/50.8 MB 11.2 MB/s eta 0:00:03
   ---------------- ----------------------- 21.0/50.8 MB 11.1 MB/s eta 0:00:03
   ------------------ --------------------- 23.3/50.8 MB 11.2 MB/s eta 0:00:03
   -------------------- ------------------- 25.7/50.8 MB 11.2 MB/s eta 0:00:03
   ---------------------- ----------------- 28.0/50.8 MB 11.3 MB/

creating json  (1)

In [6]:
import cv2
import mediapipe as mp
import json


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


cap = cv2.VideoCapture('squats.MOV')

correct_form = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # Draw the pose landmarks on the frame
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        
        frame_landmarks = []
        for landmark in results.pose_landmarks.landmark:
            frame_landmarks.append({
                'x': landmark.x,
                'y': landmark.y,
                'z': landmark.z
            })
        correct_form.append(frame_landmarks)

    
    cv2.imshow('Capture Correct Form', frame)
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save the JSON file
with open('squats.json', 'w') as json_file:
    json.dump(correct_form, json_file, indent=4)

print("Correct form saved to 'correct_form.json'")


Correct form saved to 'correct_form.json'


Procssing the pre uploaded video with landmarks and skeleton (2)

In [5]:
import cv2
import mediapipe as mp


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils


def draw_landmarks(frame, landmarks, thickness=8):
    height, width, _ = frame.shape
    for landmark in landmarks.landmark:
        x, y = int(landmark.x * width), int(landmark.y * height)
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1) 
    drawing_spec = mp_drawing.DrawingSpec(thickness=thickness, circle_radius=5)  
    
    mp_drawing.draw_landmarks(frame, landmarks, mp_pose.POSE_CONNECTIONS, drawing_spec, drawing_spec)

# Load the video of the correct yoga pose
correct_pose_video = 'vid.MOV'
cap = cv2.VideoCapture(correct_pose_video)

# Define the codec and create a VideoWriter object to save the processed video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('sitting.avi', fourcc, cap.get(cv2.CAP_PROP_FPS), 
                      (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB for MediaPipe processing
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Draw landmarks on the frame
        draw_landmarks(frame, results.pose_landmarks)

    # Write the processed frame to the video file
    out.write(frame)

    # Display the video feed with landmarks
    cv2.imshow('Correct Pose with Landmarks', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


live video processing  (old)  (3)

In [3]:
import cv2
import mediapipe as mp
import json
import numpy as np


with open('correct_form.json', 'r') as json_file:
    correct_form = json.load(json_file)


mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Set up thresholds for feedback
threshold = 0.4  
highlight_threshold = 0.25 

# Function to calculate joint errors between live frame and correct form
def calculate_joint_errors(live_frame, correct_frame):
    joint_errors = []
    for i in range(len(live_frame)):
        error = np.sqrt(np.sum((np.array([live_frame[i]['x'], live_frame[i]['y'], live_frame[i]['z']]) -
                                np.array([correct_frame[i]['x'], correct_frame[i]['y'], correct_frame[i]['z']])) ** 2))
        joint_errors.append(error)
    return joint_errors

# Open the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB for MediaPipe processing
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Extract pose landmarks from the live frame
        live_frame_landmarks = []
        for landmark in results.pose_landmarks.landmark:
            live_frame_landmarks.append({'x': landmark.x, 'y': landmark.y, 'z': landmark.z})


        for correct_frame in correct_form:
            joint_errors = calculate_joint_errors(live_frame_landmarks, correct_frame)

        
        mp_drawing.draw_landmarks(
            frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=3, circle_radius=3), 
        )

        # Visual feedback for each joint based on joint_errors
        for idx, joint_error in enumerate(joint_errors):
            height, width, _ = frame.shape
            cx, cy = int(results.pose_landmarks.landmark[idx].x * width), int(results.pose_landmarks.landmark[idx].y * height)

            if joint_error > threshold:
                color = (0, 0, 255)  # Red 
            elif joint_error <= highlight_threshold:
                color = (0, 255, 0)  # Green 
            else:
                color = (255, 255, 255)  # White 

            cv2.circle(frame, (cx, cy), 3, color, -1)  

        # Display overall pose feedback
        overall_error = np.mean(joint_errors)
        feedback_text = "Pose matches!" if overall_error < threshold else f"Adjust your pose! Error: {overall_error:.4f}"
        cv2.putText(frame, feedback_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Resize the frame to make it larger
    scale_factor = 1.5  # Adjust this scale factor as needed
    new_width = int(frame.shape[1] * scale_factor)
    new_height = int(frame.shape[0] * scale_factor)
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Display the resized frame
    cv2.imshow('Live Yoga Pose with Feedback', resized_frame)

    # Exit on pressing 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'correct_form.json'

latest live video tracking  (4)

In [11]:
import cv2
import mediapipe as mp
import json
import numpy as np



mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=0, enable_segmentation=False)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles




with open('sitting.json', 'r') as json_file:
    correct_form = json.load(json_file)




threshold = 0.4  
highlight_threshold = 0.25  





def calculate_joint_errors(live_frame, correct_frame):
    joint_errors = []
    for i in range(len(live_frame)):
        error = np.sqrt(np.sum((np.array([live_frame[i]['x'], live_frame[i]['y'], live_frame[i]['z']]) -
                                np.array([correct_frame[i]['x'], correct_frame[i]['y'], correct_frame[i]['z']])) ** 2))
        joint_errors.append(error)
    return joint_errors

# Function to draw connections in red if landmarks are faulty
def draw_skeleton_with_feedback(frame, landmarks, joint_errors, threshold):
    height, width, _ = frame.shape
    faulty_landmarks = []

    # Define connections between landmarks (based on MediaPipe Pose)
    connections = mp_pose.POSE_CONNECTIONS

    # Convert landmarks to pixel coordinates
    pixel_landmarks = [(int(l.x * width), int(l.y * height)) for l in landmarks]

    # Mark faulty landmarks
    for idx, joint_error in enumerate(joint_errors):
        if joint_error > threshold:
            faulty_landmarks.append(idx)

    # Draw connections and highlight faulty ones
    for connection in connections:
        start_idx, end_idx = connection

        start_point = pixel_landmarks[start_idx]
        end_point = pixel_landmarks[end_idx]

        # Check if either of the landmarks in the connection is faulty
        if start_idx in faulty_landmarks or end_idx in faulty_landmarks:
            color = (0, 0, 255)  # Red for faulty connections
        else:
            color = (0, 255, 0)  # Green for correct connections

        # Draw the connection line
        cv2.line(frame, start_point, end_point, color, 3)

    # Draw circles for each landmark (green for correct, red for faulty)
    for idx, (cx, cy) in enumerate(pixel_landmarks):
        color = (0, 255, 0) if idx not in faulty_landmarks else (0, 0, 255)
        cv2.circle(frame, (cx, cy), 5, color, -1)




cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        live_frame_landmarks = []
        for landmark in results.pose_landmarks.landmark:
            live_frame_landmarks.append({'x': landmark.x, 'y': landmark.y, 'z': landmark.z})
        
        for correct_frame in correct_form:
            joint_errors = calculate_joint_errors(live_frame_landmarks, correct_frame)
            
            # Draw visual feedback (skeleton with faulty landmarks in red)
            draw_skeleton_with_feedback(frame, results.pose_landmarks.landmark, joint_errors, threshold)

            # Overall Pose Feedback
            overall_error = np.mean(joint_errors)
            feedback_text = "Pose matches!" if overall_error < threshold else f"Adjust your pose! Error: {overall_error:.4f}"
            cv2.putText(frame, feedback_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    # Resize the frame to make it larger
    scale_factor = 1.5  # Adjust this scale factor as needed
    new_width = int(frame.shape[1] * scale_factor)
    new_height = int(frame.shape[0] * scale_factor)
    resized_frame = cv2.resize(frame, (new_width, new_height))

    # Display the resized frame
    cv2.imshow('Live Yoga Pose with Feedback', resized_frame)

    # Check for the 'q' key press to quit
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

# Release resources and close windows
cap.release()
cv2.destroyAllWindows()
